In [4]:
import math

def monitoring_metrics(system_type, y_true, y_pred):
    n = len(y_true)

    def safe_div(num, den):
        return num / den if den != 0 else 0.0

    metrics = {}

    if system_type == "classification":
        tp = fp = fn = tn = 0

        for yt, yp in zip(y_true, y_pred):
            if yt == 1 and yp == 1:
                tp += 1
            elif yt == 0 and yp == 1:
                fp += 1
            elif yt == 1 and yp == 0:
                fn += 1
            elif yt == 0 and yp == 0:
                tn += 1

        accuracy = safe_div(tp + tn, n)
        precision = safe_div(tp, tp + fp)
        recall = safe_div(tp, tp + fn)
        f1 = safe_div(2 * precision * recall, precision + recall)

        metrics["accuracy"] = accuracy
        metrics["precision"] = precision
        metrics["recall"] = recall
        metrics["f1"] = f1

    elif system_type == "regression":
        if n == 0:
            metrics["mae"] = 0.0
            metrics["rmse"] = 0.0
        else:
            abs_err_sum = 0.0
            sq_err_sum = 0.0

            for yt, yp in zip(y_true, y_pred):
                diff = yt - yp
                abs_err_sum += abs(diff)
                sq_err_sum += diff * diff

            mae = abs_err_sum / n
            rmse = math.sqrt(sq_err_sum / n)

            metrics["mae"] = mae
            metrics["rmse"] = rmse

    elif system_type == "ranking":
        # Sort items by predicted score descending
        items = list(zip(y_true, y_pred))
        items.sort(key=lambda x: x[1], reverse=True)

        top3 = items[:3]
        relevant_in_top3 = sum(1 for yt, _ in top3 if yt == 1)
        total_relevant = sum(1 for yt in y_true if yt == 1)

        precision_at_3 = safe_div(relevant_in_top3, 3)
        recall_at_3 = safe_div(relevant_in_top3, total_relevant)

        metrics["precision_at_3"] = precision_at_3
        metrics["recall_at_3"] = recall_at_3

    else:
        raise ValueError("system_type must be one of: classification, regression, ranking")

    return sorted(metrics.items(), key=lambda x: x[0])
